In [160]:
%load_ext autoreload
%autoreload 2

import sys  
import importlib
import os
sys.path.insert(0, '/Users/urimarom/PycharmProjects/fossil_classification')
from reports_etl import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = "{:,.2f}".format

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [161]:
last_updated()

.py file last modified: 2021-11-30 02:54:40.558650


# Bulk download reports
Using the http response recorded while searching for reports here:
https://employersinfocmp.cma.gov.il/#/publicreports

In [139]:
# get reports from response.json search results
response_path = "/Users/urimarom/PycharmProjects/fossil_classification/data/downloaded reports/company reports/"
reports = get_reports_from_response(response_path)

In [ ]:
# download reports - run only once per response
download_reports(reports, response_path)

# Process multiple reports -> unified file

In [6]:
reports_fn_list = get_filename_list(response_path)

number of files to be pre-processed: 404


## Pre-process - check sheet names and column names, add to standardization if needed

In [ ]:
pre_process_reports(reports_fn_list)

# process holding reports
1. summary sheets (for verification)
2. holding sheets - raw data

## 1. summary sheets

In [7]:
all_summary_sheets = process_summary_sheets(reports_fn_list)

In [8]:
totals = get_totals(all_summary_sheets)

Number of totals found: 403


In [9]:
totals["sum_num"].sum()

8512255491.542575

There is a difference, but not a huge one. Looking for reports with missing sum below.

## 2. Holdings

In [30]:
# extract all holdings from reports
all_holdings_clean = clean_holdings(extract_holdings(reports_fn_list))

Processing report 404 out of 404
before cleaning: 561135
 after cleaning: 501098


# add fund and company data to holdings

In [31]:
## Manually add missing corps to reports - only if needed
# missing_corps = pd.DataFrame(columns=reports_df.columns)
# missing_corps = missing_corps.append({'SystemName': 'חיים ואובדן כושר עבודה',
#                                       'ParentCorpName': 'מנורה מבטחים ביטוח בע"מ',
#                                       'ParentCorpLegalId': '520042540',
#                                       'ReportPeriodDesc': '2021 רבעון 2'
#                                      }, ignore_index=True)
# missing_corps = missing_corps.append({'SystemName': 'חיים ואובדן כושר עבודה',
#                                       'ParentCorpName': 'כלל חברה לביטוח בע"מ',
#                                       'ParentCorpLegalId': '520024647',
#                                       'ReportPeriodDesc': '2021 רבעון 2'
#                                      }, ignore_index=True)

# reports = reports.append(missing_corps)

In [35]:
all_holdings_final = add_report_data(all_holdings_clean, reports)

In [36]:
all_holdings_final.to_csv("all_holdings.csv", index=False)

# Validate holdings file

In [37]:
all_holdings_2021q2 = all_holdings_final[all_holdings_final["ReportPeriodDesc"] == '2021 רבעון 2']
pd.DataFrame(all_holdings_2021q2.groupby(['SystemName', 'ParentCorpName', 'report_id'])['שווי'].agg('sum').map('{:,.2f}'.format))

שווי
SystemName ParentCorpName                                     report_id                          
ביטוח      איי. די. איי. חברה לביטוח בע"מ                     2101093                  301,460.59
           איילון חברה לביטוח בע"מ                            2104169                4,827,252.48
           הפניקס חברה לביטוח בע"מ                            2106276               72,073,351.75
           הראל חברה לביטוח בע"מ                              2104421               70,154,192.13
           כלל חברה לביטוח בע"מ                               520024647_bsum_0221   85,330,648.82
           מגדל חברה לביטוח בע"מ                              2101872              135,461,926.93
           מנורה מבטחים ביטוח בע"מ                            520042540_bsum_p221   34,036,324.82
גמל        אינפיניטי ניהול השתלמות וגמל בע"מ                  2108839                1,379,417.06
           אלטשולר שחם גמל ופנסיה בע"מ                        2105697              169,038,676.19
           אנליסט קופות גמל בע"מ                              2082494               11,978,311.77
           ארם גמולים - חברה לניהול קופות גמל בע''מ           2100948                1,223,994.60
           גל -ניהול קופות גמל לעובדי הוראה בע"מ              2116604                4,629,693.43
           החברה המנהלת של מינהל קרן ההשתלמות לפקידים עובד... 2117779                  893,777.33
           החברה המנהלת של קרן השתלמות של עובדי חברת החשמל... 2114682                  729,255.23
           החברה המנהלת של רום קרן ההשתלמות לעובדי הרשויות... 2101356                4,049,229.31
           החברה לניהול קופות התגמולים והפיצויים של עובדי ... 2085301                5,370,349.57
           החברה לניהול קרן ההשתלמות להנדסאים וטכנאים בע"מ    2118005                2,150,673.46
           החברה לניהול קרן ההשתלמות לעובדי המדינה בע"מ       2082653                2,601,564.36
           החברה לניהול קרן השתלמות לביוכימאים  ומקרוביולו... 2102634                  320,077.68
           החברה לניהול קרן השתלמות למשפטנים בע"מ             2119050                  412,185.07
           הלמן - אלדובי קופות גמל ופנסיה בע"מ                2106482               16,136,063.02
           הנדסאים וטכנאים - חברה לניהול קופות גמל בע"מ       2118332                1,119,080.11
           הפניקס פנסיה וגמל בע"מ                             2106362               38,450,557.13
           הראל פנסיה וגמל בע"מ                               2104350               39,429,128.09
           חברה לניהול קופות גמל של העובדים בעיריית תל - א... 2093283                  152,292.54
           חברת הגמל לעובדי האוניברסיטה העברית בע"מ           2084711                2,543,163.36
           יהב - פ.ר.ח. - חברה לניהול קופות גמל בע"מ          2098739                  181,894.19
           יהב אחים ואחיות - חברה לניהול קופות גמל בע"מ       2098696                2,281,248.07
           יהב רופאים - חברה לניהול קופות גמל בע"מ            2084771                1,335,735.29
           ילין לפידות ניהול קופות גמל בע"מ                   513611509_gsum_p221   44,839,306.90
           כלל פנסיה וגמל בע"מ                                2116272               41,307,254.60
           לאומי קמ"פ בע"מ                                    2083669                  142,893.31
           מגדל מקפת קרנות פנסיה וקופות גמל בע"מ              2097413               20,603,780.67
           מור קופות גמל בע"מ                                 2100922               19,956,044.26
           מחוג - מינהל גמל לעובדי חברת חשמל לישראל בע"מ      2096890                2,738,438.59
           מחר - חברה לניהול קופות גמל בע"מ                   2103729                1,359,906.77
           מיטב דש גמל ופנסיה בע"מ                            2103793               44,297,741.08
           מנורה מבטחים והסתדרות המהנדסים ניהול קופות גמל ... 2116914                2,427,500.37
           מנורה מבטחים פנסיה וגמל בע"מ                       2117052               31,119,507.87
           סלייס גמל בע"מ                                     2085195      

# Comparing totals from summary sheets with totals from holdings

In [38]:
print("total from summary sheets: {:,.2f}".format(totals["sum_num"].sum()))
print("total from holdings sheets: {:,.2f}".format(all_holdings_final["שווי"].sum()))
print("diff: {:,.2f}".format(totals["sum_num"].sum() - all_holdings_final["שווי"].sum()))

total from summary sheets: 8,512,255,491.54
total from holdings sheets: 8,506,643,221.85
diff: 5,612,269.70


In [39]:
totals_from_holdings = pd.DataFrame(all_holdings_final.groupby("report_id")["שווי"].agg('sum')).reset_index()
totals_from_holdings.index = totals_from_holdings.index.astype('str')

In [40]:
totals_comparison = totals_from_holdings.merge(totals,
                                               on='report_id',
                                               how='outer'
                                              )

totals_comparison["diff"] = totals_comparison["sum_num"] - totals_comparison["שווי"]
totals_comparison[totals_comparison["diff"] > 1].sort_values("diff", ascending=False)

,report_id,שווי,asset,sum,pct,pct_num,sum_num,diff
385,2106364,"36,592,810.71","סה""כ סכום נכסי המסלול או הקרן","42,061,904.82",1,1.00,"42,061,904.82","5,469,094.11"
400,513026484_psum_p121,"37,377,448.67","סה""כ סכום נכסי המסלול או הקרן","38,610,977.17",1,1.00,"38,610,977.17","1,233,528.50"
109,1883389,"34,468,877.12","סה""כ סכום נכסי המסלול או הקרן","35,231,476.66",1,1.00,"35,231,476.66","762,599.54"
59,1840271,"33,007,954.10","סה""כ סכום נכסי המסלול או הקרן","33,764,881.11",1.00,1.00,"33,764,881.11","756,927.00"
110,1883536,"68,754,460.48","סה""כ סכום נכסי המסלול או הקרן","69,227,071.10",1,1.00,"69,227,071.10","472,610.62"
58,1840225,"64,633,877.81","סה""כ סכום נכסי המסלול או הקרן","65,049,264.72",1.00,1.00,"65,049,264.72","415,386.90"
108,1883258,"65,081,765.31","סה""כ סכום נכסי המסלול או הקרן","65,213,985.49",1.00,1.00,"65,213,985.49","132,220.19"
60,1840306,"60,813,009.73","סה""כ סכום נכסי המסלול או הקרן","60,942,804.62",1.00,1.00,"60,942,804.62","129,794.89"
15,1824549,"1,625,518.40","סה""כ סכום נכסי הקופה","1,696,813.00",1,1.00,"1,696,813.00","71,294.60"
247,2019519,"35,591,906.62","סה""כ סכום נכסי המסלול או הקרן","35,592,703.66",1,1.00,"35,592,703.66",797.04


2106364 - הפניקס
סכום המזומנים לא שווה לסכום של הנכסים בפועל - הרבה מהם בעלי שווי 0, כנראה באג במקור

## Look for missing company reports

In [41]:
reports_cnt = pd.DataFrame(all_holdings_final.groupby(['SystemName', 'ParentCorpName', 'ParentCorpLegalId'])['ReportPeriodDesc'].nunique().reset_index())
period_cnt = all_holdings_final["ReportPeriodDesc"].nunique()

companies_with_missing_reports = reports_cnt[reports_cnt["ReportPeriodDesc"] < period_cnt]

In [42]:
available_reports_periods = pd.DataFrame(all_holdings_final.groupby(['SystemName', 'ParentCorpName', 'ParentCorpLegalId', 'ReportPeriodDesc'])['שווי'].sum()).reset_index()
companies_w_missing_reports_available_reports = pd.merge(
    companies_with_missing_reports[['ParentCorpLegalId', 'SystemName']],
    available_reports_periods,
    on = ['ParentCorpLegalId', 'SystemName'],
    how="inner"
)
companies_w_missing_reports_available_reports

,ParentCorpLegalId,SystemName,ParentCorpName,ReportPeriodDesc,שווי
0,520042177,ביטוח,"הכשרה חברה לביטוח בע""מ",2020 רבעון 1,"14,286,995.88"
1,520042177,ביטוח,"הכשרה חברה לביטוח בע""מ",2020 רבעון 2,"15,662,871.06"
2,520042177,ביטוח,"הכשרה חברה לביטוח בע""מ",2020 רבעון 3,"16,923,932.26"
3,520042177,ביטוח,"הכשרה חברה לביטוח בע""מ",2021 רבעון 1,"21,250,381.04"
4,520034968,גמל,החברה המנהלת של קרן השתלמות של עובדי חברת החשמ...,2021 רבעון 1,"730,533.13"
5,520034968,גמל,החברה המנהלת של קרן השתלמות של עובדי חברת החשמ...,2021 רבעון 2,"729,255.23"
6,520005497,גמל,החברה לניהול קופות התגמולים והפיצויים של עובדי...,2020 רבעון 3,"4,972,323.19"
7,520005497,גמל,החברה לניהול קופות התגמולים והפיצויים של עובדי...,2020 רבעון 4,"5,105,050.69"
8,520005497,גמל,החברה לניהול קופות התגמולים והפיצויים של עובדי...,2021 רבעון 1,"5,263,096.89"
9,520005497,גמל,החברה לניהול קופות התגמולים והפיצויים של עובדי...,2021 רבעון 2,"5,370,349.57"


Located missing reports for major companies, manually downloaded

# Add fossil classification
Based on quarterly company level reports classification

## get previous fossil classifications

In [118]:
prev_cls_fn = "/Users/urimarom/PycharmProjects/fossil_classification/data/חשיפה לפוסיליים - סיווגי רבעונים קודמים - all up to 2021Q2.csv"
prev_cls = get_latest_fossil_classifications(prev_cls_fn)

previously classified by is_fossil:
0.00    6938
1.00     568
Name: is_fossil, dtype: int64


## directly using previous company reports
Could be useful for fund level reports, commented out for now

In [119]:
# # TODO: using different file per holding_type for now, should change
# q_company_report1 = pd.read_csv('/Users/urimarom/Downloads/חשיפה לפוסיליים - Q2 2021 - החזקות ישירות באגח קונצרני.csv')
# q_company_report2 = pd.read_csv('/Users/urimarom/Downloads/חשיפה לפוסיליים - Q2 2021 - החזקות ישירות במניות.csv')

# q_company_report = pd.concat([
#     q_company_report1[['מספר ני"ע', 'is_fossil']],
#     q_company_report2[['מספר ני"ע', 'is_fossil']]
# ])

# q_company_report = q_company_report.drop_duplicates(subset=['מספר ני"ע']).set_index('מספר ני"ע')
# q_company_report.index = q_company_report.index.astype('str')

## Using previous classification

In [130]:
all_holdings_cls = add_fossil_classifications(all_holdings_final, prev_cls)

all_holdings: 501098
having holding number: 499096
without holding number: 2002


/Users/urimarom/PycharmProjects/fossil_classification/reports_etl.py:476: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fossil_cls.index = fossil_cls.index.astype('str')


Holdings by fossil classification:
nan     307706
0.00    171171
1.00     20219
Name: is_fossil, dtype: int64
total fossil sum: 283012033.304952
holdings count before classification: 501098
holdings count after classification: 501098


In [131]:
all_holdings_cls.to_csv("all_holdings_with_cls.csv", index=False)

## Validate Fossil Classification results

In [132]:
# look for fossil holdings that aren't מניות or אג"ח קונצרני
all_holdings_cls[
    (all_holdings_cls["is_fossil"] > 0) &
    (~all_holdings_cls["holding_type"].isin(["מניות", 'אג"ח קונצרני']))
]

# looking good!
# TODO: might want to try and catch additional fossil holdings of other types

,שם המנפיק/שם נייר ערך,"מספר ני""ע",מספר מנפיק,דירוג,שם מדרג,סוג מטבע,שעור ריבית,תשואה לפדיון,שווי,שעור מנכסי אפיק ההשקעה,שעור מסך נכסי השקעה,report_id,holding_type,זירת מסחר,תאריך רכישה,"מח""מ",ערך נקוב,שער,פדיון/ריבית/דיבידנד לקבל,שעור מערך נקוב מונפק,ספק מידע,ענף מסחר,נכס בסיס,קונסורציום כן/לא,תאריך שערוך אחרון,אופי הנכס,שעור תשואה במהלך התקופה,כתובת הנכס,ריבית אפקטיבית,SystemName,ParentCorpName,ParentCorpLegalId,ProductNum,Name,ShortName,StatusDate,ReportPeriodDesc,is_fossil,שווי פוסילי
6331,נאוויטס פט אפ 4,1156454,NaN,NaN,NaN,שקל חדש,NaN,NaN,71.29,0.41,0,1823174,כתבי אופציה,TASE,NaN,NaN,36750,194,NaN,0.03,NaN,חיפושי נפט וגז,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,החברה המנהלת של רום קרן ההשתלמות לעובדי הרשויו...,520031824,0.00,nan,nan,25/05/2020,2020 רבעון 1,1.00,71.29
9490,רציו אפ 19,3940319,NaN,NaN,NaN,שקל חדש,NaN,NaN,32.16,0.00,0.00,2056812,כתבי אופציה,TASE,NaN,NaN,"417,634.56",7.70,NaN,0.00,NaN,חיפושי נפט וגז,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ביטוח,"הראל חברה לביטוח בע""מ",520004078,0.00,nan,nan,09/06/2021,2021 רבעון 1,1.00,32.16
12104,נאוויטס פט אפ 4,1156454,NaN,NaN,NaN,שקל חדש,NaN,NaN,8.55,0.01,0,1944444,כתבי אופציה,TASE,NaN,NaN,23300,36.70,NaN,0.02,NaN,חיפושי נפט וגז,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,ארם גמולים - חברה לניהול קופות גמל בע''מ,510773922,0.00,nan,nan,29/11/2020,2020 רבעון 3,1.00,8.55
12105,רותם אנרגיה אפ1,1155720,NaN,NaN,NaN,שקל חדש,NaN,NaN,0.20,0.00,0,1944444,כתבי אופציה,TASE,NaN,NaN,9050,2.20,NaN,0.00,NaN,חיפושי נפט וגז,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,ארם גמולים - חברה לניהול קופות גמל בע''מ,510773922,0.00,nan,nan,29/11/2020,2020 רבעון 3,1.00,0.20
12106,רותם אנרגיה אפ2,1155738,NaN,NaN,NaN,שקל חדש,NaN,NaN,0.85,0.00,0,1944444,כתבי אופציה,TASE,NaN,NaN,7240,11.70,NaN,0.00,NaN,חיפושי נפט וגז,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,ארם גמולים - חברה לניהול קופות גמל בע''מ,510773922,0.00,nan,nan,29/11/2020,2020 רבעון 3,1.00,0.85
12107,רותם אנרגיה אפ3,1155746,NaN,NaN,NaN,שקל חדש,NaN,NaN,0.31,0.00,0,1944444,כתבי אופציה,TASE,NaN,NaN,1810,17,NaN,0.00,NaN,חיפושי נפט וגז,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,ארם גמולים - חברה לניהול קופות גמל בע''מ,510773922,0.00,nan,nan,29/11/2020,2020 רבעון 3,1.00,0.31
13956,XOP US,US78468R5569,513834200,NaN,NaN,דולר אמריקאי,NaN,NaN,"20,532.32",0.01,0.00,2106482,קרנות סל,NASDAQ,NaN,NaN,"65,138.67",9669,0,0.00,NaN,מניות,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"הלמן - אלדובי קופות גמל ופנסיה בע""מ",512227265,0.00,nan,nan,09/09/2021,2021 רבעון 2,1.00,"20,532.32"
14052,רציו אפ 19,3940319,NaN,NaN,NaN,שקל חדש,NaN,NaN,5.99,0.00,0,2106482,כתבי אופציה,TASE,NaN,NaN,598810,1,NaN,0.01,NaN,חיפושי נפט וגז,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"הלמן - אלדובי קופות גמל ופנסיה בע""מ",512227265,0.00,nan,nan,09/09/2021,2021 רבעון 2,1.00,5.99
16481,רותם אנרגיה אפ2,1155738,NaN,NaN,NaN,שקל חדש,NaN,NaN,0.06,0.00,0,2051791,כתבי אופציה,TASE,NaN,NaN,920,6.80,NaN,0.00,NaN,חיפושי נפט וגז,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"החברה לניהול קרן השתלמות למשפטנים בע""מ",520028861,0.00,nan,nan,31/05/2021,2021 רבעון 1,1.00,0.06
16482,רותם אנרגיה אפ3,1155746,NaN,NaN,NaN,שקל חדש,NaN,NaN,0.03,0.00,0,2051791,כתבי אופציה,TASE,NaN,NaN,230,13.50,NaN,0.00,NaN,חיפושי נפט וגז,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"החברה לניהול קרן השתלמות למשפטנים בע""מ",520028861,0.00,nan,nan,31/05/2021,2021 רבעון 1,1.00,0.03


In [133]:
should_be_cls = all_holdings_cls[all_holdings_cls["holding_type"].isin(["מניות", 'אג"ח קונצרני'])]
missing_fossil_cls = pd.DataFrame(
    should_be_cls["is_fossil"].isnull().groupby([should_be_cls["ParentCorpName"], should_be_cls["ReportPeriodDesc"]]).sum()
)
missing_fossil_cls

is_fossil
ParentCorpName                                     ReportPeriodDesc           
איי. די. איי. חברה לביטוח בע"מ                     2020 רבעון 1              6
                                                   2020 רבעון 2              2
                                                   2020 רבעון 3              2
                                                   2020 רבעון 4              2
                                                   2021 רבעון 1              2
                                                   2021 רבעון 2              4
איילון חברה לביטוח בע"מ                            2020 רבעון 1             29
                                                   2020 רבעון 2             18
                                                   2020 רבעון 3             18
                                                   2020 רבעון 4             18
                                                   2021 רבעון 1             21
                                                   2021 רבעון 2             19
אינפיניטי ניהול השתלמות וגמל בע"מ                  2020 רבעון 1              8
                                                   2020 רבעון 2              1
                                                   2020 רבעון 3              0
                                                   2020 רבעון 4              0
                                                   2021 רבעון 1              2
                                                   2021 רבעון 2              3
אלטשולר שחם גמל ופנסיה בע"מ                        2020 רבעון 1             16
                                                   2020 רבעון 2              0
                                                   2020 רבעון 3              0
                                                   2020 רבעון 4              0
                                                   2021 רבעון 1              0
                                                   2021 רבעון 2              0
אנליסט קופות גמל בע"מ                              2020 רבעון 1             26
                                                   2020 רבעון 2             13
                                                   2020 רבעון 3             10
                                                   2020 רבעון 4              7
                                                   2021 רבעון 1              4
                                                   2021 רבעון 2              5
ארם גמולים - חברה לניהול קופות גמל בע''מ           2020 רבעון 1             13
                                                   2020 רבעון 2              5
                                                   2020 רבעון 3              4
                                                   2020 רבעון 4              5
                                                   2021 רבעון 1              8
                                                   2021 רבעון 2              8
גל -ניהול קופות גמל לעובדי הוראה בע"מ              2020 רבעון 1             19
                                                   2020 רבעון 2              5
                                                   2020 רבעון 3              3
                                                   2020 רבעון 4              4
                                                   2021 רבעון 1              8
                                                   2021 רבעון 2             10
החברה המנהלת של מינהל קרן ההשתלמות לפקידים עובד... 2020 רבעון 1              0
                                                   2020 רבעון 2              2
                                                   2020 רבעון 3              0
                                                   2020 רבעון 4              0
                                                   2021 רבעון 1              1
                                                   2021 רבעון 2              0
החברה המנהלת של קרן השתלמות של עובדי חברת החשמל... 2021 רבעון 1             10
                                        

# Let the Analysis begin!

In [135]:
pd.DataFrame(all_holdings_cls.groupby(['SystemName'])['שווי'].agg('sum').map('{:,.2f}'.format))

,שווי
SystemName,
ביטוח,"2,220,079,121.25"
גמל,"3,259,601,753.75"
פנסיה,"3,026,962,346.85"


In [145]:
def get_summary(arg1, *args, holdings=all_holdings_cls):
    '''get summary stats grouped by 1 or more columns, e.g. Company, holding_type
    '''
    group = [arg1]+[*args]
    summary = pd.DataFrame(holdings.groupby(group, dropna=False).agg(
    {'שווי':'sum', 'שווי פוסילי':'sum'})).reset_index()
    summary["שיעור פוסילי מסך הנכסים"] = 100 * summary["שווי פוסילי"] / summary["שווי"]
    # add summary of stocks and bonds only
    holdings_cls_trd_stocks_bonds = holdings[holdings["holding_type"].isin(['מניות', 'אג"ח קונצרני'])]
    summary_tradable_stocks_bonds_only = pd.DataFrame(holdings_cls_trd_stocks_bonds.groupby(group, dropna=False).agg(
    {'שווי':'sum', 'שווי פוסילי':'sum'})).reset_index()
    summary_tradable_stocks_bonds_only["שיעור פוסילי במניות ואגח קונצרני סחירים"] = 100 * summary_tradable_stocks_bonds_only[
        "שווי פוסילי"] / summary_tradable_stocks_bonds_only["שווי"]
    summary = summary.merge(summary_tradable_stocks_bonds_only,
                            left_on=group,
                            right_on=group,
                            how='left',
                            suffixes = ['', ' במניות ואגח קונצרני סחירים']
                           )
    return summary

## Get Company level stats over time

### Filter results to include only major companies

In [172]:
institutions = [
    'מנורה',
    'מגדל',
    'הראל',
    'מיטב דש',
    'אלטשולר',
    'מור',
    'הפניקס',
    'כלל',
    'פסגות',
    'הלמן',
    'אנליסט',
    'ילין'
]

mask = all_holdings_cls["ParentCorpName"].str.startswith(tuple(institutions))

# removing הלמן-אלדובי חח"י גמל בע"מ 515447035 & 520027715, מנורה מבטחים והסתדרות המהנדסים ניהול קופות גמל בע"מ
all_holdings_cls_filtered = all_holdings_cls[mask][~all_holdings_cls[mask]["ParentCorpLegalId"].isin(['520027715', '515447035'])]
# all_holdings_cls_filtered.groupby(["ParentCorpName","ParentCorpLegalId"]).size()

all_holdings_cls_filtered['ParentCorpGroup'] = all_holdings_cls_filtered['ParentCorpName'].str.split().str[0]
all_holdings_cls_filtered['ReportPeriodDate'] = all_holdings_cls_filtered['ReportPeriodDesc'].map(report_period_desc_to_date)
company_stats_since_2020 = get_summary('ReportPeriodDate', 'ParentCorpGroup', holdings=all_holdings_cls_filtered)

In [173]:
company_stats_since_2020.to_csv("company_stats_since_2020.csv", index=False)

## Get Fund stats

In [395]:
fund_stats = get_summary('ParentCorpName', 'SystemName', 'ProductNum','ShortName')
fund_stats.sort_values('שיעור פוסילי מסך הנכסים')

,ParentCorpName,SystemName,ProductNum,ShortName,שווי,שווי פוסילי,שיעור פוסילי מסך הנכסים,שווי במניות ואגח קונצרני סחירים,שווי פוסילי במניות ואגח קונצרני סחירים,שיעור פוסילי במניות ואגח קונצרני סחירים
800,"פסגות קופות גמל ופנסיה בע""מ",גמל,11383,פסגות חיסכון לילד - הלכה,"1,142,471.17",0.00,0.00,nan,nan,nan
517,"מגדל מקפת קרנות פנסיה וקופות גמל בע""מ",פנסיה,8603,מקפת אישית הלכה למקבלי קצבה קיימים,504.76,0.00,0.00,nan,nan,nan
513,"מגדל מקפת קרנות פנסיה וקופות גמל בע""מ",פנסיה,2149,מקפת משלימה הלכה,"5,580.40",0.00,0.00,nan,nan,nan
511,"מגדל מקפת קרנות פנסיה וקופות גמל בע""מ",פנסיה,2147,מקפת משלימה שקלי טווח קצר,"2,528.74",0.00,0.00,nan,nan,nan
507,"מגדל מקפת קרנות פנסיה וקופות גמל בע""מ",פנסיה,2143,מגדל מקפת אישית שקלי טווח קצר,"169,198.50",0.00,0.00,nan,nan,nan
505,"מגדל מקפת קרנות פנסיה וקופות גמל בע""מ",פנסיה,2112,מגדל מקפת אישית הלכה,"1,132,152.25",0.00,0.00,nan,nan,nan
163,"הפניקס פנסיה וגמל בע""מ",גמל,715,אקסלנס השתלמות שקלי טווח קצר,"226,210.52",0.00,0.00,"2,191.90",0.00,0.00
503,"מגדל מקפת קרנות פנסיה וקופות גמל בע""מ",גמל,13565,מחקה מדד s&p500,"11,240.47",0.00,0.00,nan,nan,nan
165,"הפניקס פנסיה וגמל בע""מ",גמל,801,אקסלנס מרכזית לפיצויים שקלי,"24,787.09",0.00,0.00,"1,042.38",0.00,0.00
502,"מגדל מקפת קרנות פנסיה וקופות גמל בע""מ",גמל,13563,מחקה מדד s&p500,"2,600.42",0.00,0.00,nan,nan,nan


In [396]:
fund_stats.to_csv("fund_stats.csv")

In [373]:
# get fossil holdings per fund

fund_stats[fund_stats["ProductNum"].duplicated()]
# TODO: there is 1 duplicate, fund number 103 - check it later
def get_fossil_holdings(fund_id):
    fossil_holdings = all_holdings_cls[
        (all_holdings_cls["ProductNum"] == fund_id) &
        (all_holdings_cls["is_fossil"] > 0)
    ]
    return fossil_holdings.sort_values("שווי פוסילי", ascending=False)

get_fossil_holdings(11366)

,שם המנפיק/שם נייר ערך,"מספר ני""ע",מספר מנפיק,דירוג,שם מדרג,סוג מטבע,שעור ריבית,תשואה לפדיון,שווי,שעור מנכסי אפיק ההשקעה,שעור מסך נכסי השקעה,report_id,holding_type,זירת מסחר,תאריך רכישה,"מח""מ",ערך נקוב,שער,פדיון/ריבית/דיבידנד לקבל,שעור מערך נקוב מונפק,ספק מידע,ענף מסחר,נכס בסיס,קונסורציום כן/לא,תאריך שערוך אחרון,אופי הנכס,שעור תשואה במהלך התקופה,כתובת הנכס,ריבית אפקטיבית,SystemName,ParentCorpName,ParentCorpLegalId,ProductNum,Name,ShortName,StatusDate,ReportPeriodDesc,is_fossil,שווי פוסילי
528808,איי.סי.אל,281014,520027830,NaN,NaN,שקל חדש,NaN,NaN,931.63,0.03,0.01,2082486,מניות,TASE,NaN,NaN,42136,2211,0,0,אחר,"כימיה, גומי ופלסטיק",NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"אנליסט קופות גמל בע""מ",511880460,11366,אנליסט מסלולית - קופת גמל להשקעה לחיסכון ארוך ...,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון בינוני,26/07/2021,2021 רבעון 2,1.00,931.63
528800,שיכון ובינוי,1081942,520036104,NaN,NaN,שקל חדש,NaN,NaN,526.39,0.02,0.01,2082486,מניות,TASE,NaN,NaN,24971,2108,0,0.00,אחר,בנייה,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"אנליסט קופות גמל בע""מ",511880460,11366,אנליסט מסלולית - קופת גמל להשקעה לחיסכון ארוך ...,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון בינוני,26/07/2021,2021 רבעון 2,1.00,526.39
528827,פז נפט,1100007,510216054,NaN,NaN,שקל חדש,NaN,NaN,494.25,0.02,0.01,2082486,מניות,TASE,NaN,NaN,1235,40020,0,0.00,אחר,אנרגיה,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"אנליסט קופות גמל בע""מ",511880460,11366,אנליסט מסלולית - קופת גמל להשקעה לחיסכון ארוך ...,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון בינוני,26/07/2021,2021 רבעון 2,1.00,494.25
528839,אלקו החזקות,694034,520025370,NaN,NaN,שקל חדש,NaN,NaN,467.15,0.02,0.00,2082486,מניות,TASE,NaN,NaN,2408,19400,0,0.00,אחר,השקעות ואחזקות,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"אנליסט קופות גמל בע""מ",511880460,11366,אנליסט מסלולית - קופת גמל להשקעה לחיסכון ארוך ...,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון בינוני,26/07/2021,2021 רבעון 2,1.00,467.15
528881,דור אלון,1093202,520043878,NaN,NaN,שקל חדש,NaN,NaN,398.29,0.01,0.00,2082486,מניות,TASE,NaN,NaN,4291,9282,0,0.00,אחר,אנרגיה,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"אנליסט קופות גמל בע""מ",511880460,11366,אנליסט מסלולית - קופת גמל להשקעה לחיסכון ארוך ...,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון בינוני,26/07/2021,2021 רבעון 2,1.00,398.29
528661,שיכון ובינוי אגח 8,1135888,520036104,ilA,S&P מעלות,שקל חדש,0.04,0.01,354.72,0.02,0.00,2082486,"אג""ח קונצרני",TASE,22/01/18,5.36,"290,278.67",122.20,0,0.00,אחר,בנייה,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"אנליסט קופות גמל בע""מ",511880460,11366,אנליסט מסלולית - קופת גמל להשקעה לחיסכון ארוך ...,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון בינוני,26/07/2021,2021 רבעון 2,1.00,354.72
528608,חשמל אגח 27,6000210,520000472,Aa1.il,מידרוג,שקל חדש,0.04,-0.00,335.17,0.02,0.00,2082486,"אג""ח קונצרני",TASE,08/03/17,6.09,"253,170.36",132.39,0,0.00,אחר,אנרגיה,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"אנליסט קופות גמל בע""מ",511880460,11366,אנליסט מסלולית - קופת גמל להשקעה לחיסכון ארוך ...,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון בינוני,26/07/2021,2021 רבעון 2,1.00,335.17
528749,רציו מימון אגח ב,1139443,515060044,0,לא מדורג,שקל חדש,0.02,0.03,318.40,0.02,0.00,2082486,"אג""ח קונצרני",TASE,30/01/17,1.06,240614,132.33,0,0.00,אחר,חיפושי נפט וגז,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"אנליסט קופות גמל בע""מ",511880460,11366,אנליסט מסלולית - קופת גמל להשקעה לחיסכון ארוך ...,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון בינוני,26/07/2021,2021 רבעון 2,1.00,318.40
528826,בזן,2590248,520036658,NaN,NaN,שקל חדש,NaN,NaN,313.00,0.01,0.00,2082486,מניות,TASE,NaN,NaN,372619,84,0,0.00,אחר,אנרגיה,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"אנליסט קופות גמל בע""מ",511880460,11366,אנליסט מסלולית - קופת גמל להשקעה לחיסכון ארוך ...,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון בינוני,26/07/2021,2021 רבעון 2,1.00,313.00
528607,חשמל אגח 29,6000236,520000472,Aa1.il,מידרוג,שקל חדש,0.04,-0.01,307.10,0.02,0.00,2082486,"אג""ח קונצרני",TASE,24/06/19,3.84,240000,127.96,0,0.00,אחר,אנרגיה,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"אנליסט קופות גמל בע""מ",511880460,11366,אנליסט מסלולית - קופת גמל להשקעה לחיסכון ארוך ...,אנליסט חיסכון לילד - חוס

## looking into specific holdings
קרנות סל - מגדל 125

In [549]:
all_holdings_2021q2[
    (all_holdings_2021q2["holding_type"] == 'קרנות סל') &
    (all_holdings_2021q2["שם המנפיק/שם נייר ערך"].str.contains("125")) &
    (all_holdings_2021q2["שם המנפיק/שם נייר ערך"].str.contains("מגדל"))
].groupby('מספר ני"ע')["שווי"].sum()

מספר ני"ע
1150283   89,073.06
1159714   37,317.87
1174796       17.59
Name: שווי, dtype: float64

In [569]:
# החזקות של חברות בקרן עוקבת תא 125 של מגדל
cols = ["SystemName", "ParentCorpName","שווי", 'שעור מנכסי אפיק ההשקעה', "שעור מערך נקוב מונפק"]
all_holdings_2021q2[
    (all_holdings_2021q2['מספר ני"ע'] == 1150283)
][cols]

,SystemName,ParentCorpName,שווי,שעור מנכסי אפיק ההשקעה,שעור מערך נקוב מונפק
13831,גמל,"הלמן - אלדובי קופות גמל ופנסיה בע""מ",231.34,0.00,0.00
18897,פנסיה,"מיטב דש גמל ופנסיה בע""מ","16,149.28",0.01,0.01
29346,ביטוח,"איילון חברה לביטוח בע""מ",560.67,0.00,0.00
40372,גמל,"פסגות קופות גמל ופנסיה בע""מ","16,325.29",0.01,0.02
74680,גמל,"הפניקס פנסיה וגמל בע""מ","1,050.54",0.00,0.00
122078,גמל,"מיטב דש גמל ופנסיה בע""מ","25,739.15",0.01,0.02
160766,גמל,"החברה לניהול קרן ההשתלמות להנדסאים וטכנאים בע""מ","1,266.51",0.00,0.00
248891,פנסיה,"פסגות קופות גמל ופנסיה בע""מ","9,443.80",0.03,0.01
279942,גמל,"אינפיניטי ניהול השתלמות וגמל בע""מ","3,342.95",0.01,0.00
398997,גמל,"החברה לניהול קרן השתלמות למשפטנים בע""מ",134.76,0.00,0.00


In [575]:
cols = ["שם המנפיק/שם נייר ערך", 'מספר ני"ע', "שווי", "שעור מנכסי אפיק ההשקעה", "שעור מערך נקוב מונפק", "SystemName"]

all_holdings_2021q2[
    (all_holdings_2021q2['holding_type'] == 'קרנות סל') &
    (all_holdings_2021q2["ParentCorpName"].str.contains('מגדל')) &
    (~all_holdings_2021q2["שם המנפיק/שם נייר ערך"].str.contains('כש')) &
    (all_holdings_2021q2["שם המנפיק/שם נייר ערך"].str.contains('125'))
][cols]

,שם המנפיק/שם נייר ערך,"מספר ני""ע",שווי,שעור מנכסי אפיק ההשקעה,שעור מערך נקוב מונפק,SystemName
88639,הראל סל תא 125,1148899,"77,115.37",0.01,0.09,ביטוח
88644,פסגות ETF תל אביב 125,1148808,"155,533.51",0.01,0.12,ביטוח
88648,קסם תא125,1146356,"140,897.26",0.01,0.07,ביטוח
88650,תכלית תא 125,1143718,"47,261.98",0.00,0.02,ביטוח
281587,"הראל סל תא 125- הראל קרנות נאמנות בע""מ",1148899,"14,195.15",0.01,0.02,גמל
281591,"*MTF סל )4A( ת""א125- אקלים נקי- מגדל קרנות נאמ...",1174796,17.59,0,0,גמל
281596,"תכלית סל תא 125- מיטב תכלית קרנות נאמנות בע""מ",1143718,"9,595.19",0.00,0.00,גמל
281599,"פסגות ETF תא 125- פסגות קרנות מדדים בע""מ",1148808,"27,112.11",0.01,0.02,גמל
281606,"קסם ETF תא 125- קסם קרנות נאמנות בע""מ",1146356,"24,544.03",0.01,0.01,גמל
455627,הראל סל תא 125,1148899,"56,424.02",0.01,0.06,פנסיה


## Looking into ETFs

In [557]:
etf_holdings = all_holdings_2021q2[
    (all_holdings_2021q2["holding_type"] == 'קרנות סל')
]

etf_holdings.groupby("זירת מסחר")["שווי"].sum()

זירת מסחר
AMEX              118.70
ASX           483,289.25
CAC           530,710.30
DAX           121,456.62
EURONEXT    4,150,375.49
FWB         2,293,354.79
HKSE        1,230,909.60
ISE            28,392.81
JPX           187,602.86
LSE        15,767,218.60
NASDAQ      9,568,288.10
NYSE       39,434,670.96
SIX         1,638,552.83
TASE       30,436,824.11
TSE         4,318,613.35
TSX            26,454.18
אחר        13,829,003.07
Name: שווי, dtype: float64

In [562]:
etf_holdings_tase = etf_holdings[etf_holdings["זירת מסחר"] == "TASE"]
etf_holdings_tase.head()

,שם המנפיק/שם נייר ערך,"מספר ני""ע",מספר מנפיק,דירוג,שם מדרג,סוג מטבע,שעור ריבית,תשואה לפדיון,שווי,שעור מנכסי אפיק ההשקעה,שעור מסך נכסי השקעה,report_id,holding_type,זירת מסחר,תאריך רכישה,"מח""מ",ערך נקוב,שער,פדיון/ריבית/דיבידנד לקבל,שעור מערך נקוב מונפק,ספק מידע,ענף מסחר,נכס בסיס,קונסורציום כן/לא,תאריך שערוך אחרון,אופי הנכס,שעור תשואה במהלך התקופה,כתובת הנכס,ריבית אפקטיבית,SystemName,ParentCorpName,ParentCorpLegalId,ProductNum,Name,ShortName,StatusDate,ReportPeriodDesc
8585,MTF סל S&P 500 מנוטר,1150572,511303661,NaN,NaN,שקל חדש,NaN,NaN,"1,372.16",0.02,0.00,2098344,קרנות סל,TASE,NaN,NaN,23847,5754,0,0.00,NaN,מניות,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"שיבולת - חברה לניהול קופות גמל בע""מ",520030693,0,nan,nan,22/08/2021,2021 רבעון 2
8586,MTF סל indx USA clou,1158328,511303661,NaN,NaN,שקל חדש,NaN,NaN,209.26,0.00,0.00,2098344,קרנות סל,TASE,NaN,NaN,3935,5318,0,0.00,NaN,מניות,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"שיבולת - חברה לניהול קופות גמל בע""מ",520030693,0,nan,nan,22/08/2021,2021 רבעון 2
8587,הראל סל NDX 100 מנוט,1149103,511776783,NaN,NaN,שקל חדש,NaN,NaN,"1,439.72",0.02,0.00,2098344,קרנות סל,TASE,NaN,NaN,10328,13940,0,0.00,NaN,מניות,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"שיבולת - חברה לניהול קופות גמל בע""מ",520030693,0,nan,nan,22/08/2021,2021 רבעון 2
8588,הראל סל SP500 מנוטרל,1149137,511776783,NaN,NaN,שקל חדש,NaN,NaN,"1,727.06",0.02,0.00,2098344,קרנות סל,TASE,NaN,NaN,38793,4452,0,0.00,NaN,מניות,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"שיבולת - חברה לניהול קופות גמל בע""מ",520030693,0,nan,nan,22/08/2021,2021 רבעון 2
8589,מו.NDX100ממ,1165844,514884485,NaN,NaN,שקל חדש,NaN,NaN,"1,466.68",0.02,0.00,2098344,קרנות סל,TASE,NaN,NaN,19863,7384,0,0.00,NaN,מניות,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"שיבולת - חברה לניהול קופות גמל בע""מ",520030693,0,nan,nan,22/08/2021,2021 רבעון 2


## חסכון לכל ילד

In [394]:
fund_stats[fund_stats["ShortName"].str.contains("ילד", na=False)].sort_values("שיעור פוסילי במניות ואגח קונצרני סחירים", ascending=False)

,ParentCorpName,SystemName,ProductNum,ShortName,שווי,שווי פוסילי,שיעור פוסילי מסך הנכסים,שווי במניות ואגח קונצרני סחירים,שווי פוסילי במניות ואגח קונצרני סחירים,שיעור פוסילי במניות ואגח קונצרני סחירים
108,"הלמן - אלדובי קופות גמל ופנסיה בע""מ",גמל,11352,הלמן-אלדובי חיסכון לילד- שריעה,"35,974.23",949.71,2.64,"2,019.75",949.71,47.02
73,"אנליסט קופות גמל בע""מ",גמל,11366,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון בינוני,"96,700.75","8,641.30",8.94,"42,737.42","8,641.30",20.22
708,"מנורה מבטחים פנסיה וגמל בע""מ",גמל,11321,מנורה מבטחים חסכון לילד - חוסכים המעדיפים סיכו...,"327,169.05","16,185.31",4.95,"91,773.02","16,185.31",17.64
72,"אנליסט קופות גמל בע""מ",גמל,11365,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון מועט,"285,648.01","14,237.21",4.98,"88,028.15","14,237.21",16.17
709,"מנורה מבטחים פנסיה וגמל בע""מ",גמל,11322,מנורה מבטחים חסכון לילד - חוסכים המעדיפים סיכו...,"96,616.44","6,308.52",6.53,"41,211.56","6,308.52",15.31
194,"הפניקס פנסיה וגמל בע""מ",גמל,11310,אקסלנס חיסכון לילד - חוסכים המעדיפים סיכון מועט,"290,187.46","19,255.51",6.64,"136,149.36","19,255.51",14.14
110,"הלמן - אלדובי קופות גמל ופנסיה בע""מ",גמל,11354,הלמן-אלדובי חיסכון לילד- חוסכים המעדיפים סיכון...,"280,644.80","22,031.72",7.85,"160,015.44","22,031.72",13.77
111,"הלמן - אלדובי קופות גמל ופנסיה בע""מ",גמל,11355,הלמן-אלדובי חיסכון לילד- חוסכים המעדיפים סיכון...,"46,816.33","3,856.04",8.24,"28,275.67","3,856.04",13.64
196,"הפניקס פנסיה וגמל בע""מ",גמל,11312,אקסלנס חיסכון לילד - חוסכים המעדיפים סיכון בינוני,"55,268.49","3,834.87",6.94,"28,973.23","3,834.87",13.24
297,"הראל פנסיה וגמל בע""מ",גמל,11375,הראל חיסכון לילד - חוסכים המעדיפים סיכון מועט,"416,794.57","15,869.95",3.81,"123,710.98","15,867.81",12.83


In [411]:
all_holdings_cls.groupby(["SystemName","ParentCorpName","ProductNum"])["שעור מסך נכסי השקעה"].sum()

SystemName              ParentCorpName                         ProductNum
גמל                     אלטשולר שחם גמל ופנסיה בע"מ            472          1.00
                                                               558          1.00
                                                               588          1.00
                                                               1093         1.00
                                                               1094         1.00
                                                               1290         1.00
                                                               1375         1.00
                                                               1376         1.00
                                                               1377         1.00
                                                               1378         1.00
                                                               1394         1.00
                                   

## Fund level data - Discrepancies
* הלמן - seems like we looked at the wrong company level report - it was חברת חשמל...
<br>The numbers ~fit the ones in the company level PDF report. The gov site is missing a company level report.

the company name is: הלמן-אלדובי קופות גמל ופנסיה בע"מ


In [510]:
# missing reports
[r for r in funds_df.index.unique() if r not in all_holdings_final["report_id"].unique()]

# checked 2 missing, both have 0 holdings, all good

[]

In [98]:
pd.DataFrame(all_holdings_final.groupby(['ParentCorpName', 'SystemName'])["report_id"].nunique())

report_id
ParentCorpName                        SystemName                       
אלטשולר שחם גמל ופנסיה בע"מ           גמל                            30
                                      פנסיה                          14
אנליסט קופות גמל בע"מ                 גמל                            31
הלמן - אלדובי קופות גמל ופנסיה בע"מ   גמל                            39
                                      פנסיה                          12
הפניקס חברה לביטוח בע"מ               חיים ואובדן כושר עבודה         30
הפניקס פנסיה וגמל בע"מ                גמל                            46
                                      פנסיה                          24
הראל חברה לביטוח בע"מ                 חיים ואובדן כושר עבודה         34
הראל פנסיה וגמל בע"מ                  גמל                            44
                                      פנסיה                          43
ילין לפידות ניהול קופות גמל בע"מ      גמל                            20
כלל חברה לביטוח בע"מ                  חיים ואובדן כושר עבודה         21
כלל פנסיה וגמל בע"מ                   גמל                            36
                                      פנסיה                          24
מגדל חברה לביטוח בע"מ                 חיים ואובדן כושר עבודה         18
מגדל מקפת קרנות פנסיה וקופות גמל בע"מ גמל                            38
                                      פנסיה                          27
מור קופות גמל בע"מ                    גמל                            36
מיטב דש גמל ופנסיה בע"מ               גמל                            54
                                      פנסיה                          24
מנורה מבטחים ביטוח בע"מ               חיים ואובדן כושר עבודה         15
מנורה מבטחים פנסיה וגמל בע"מ          גמל                            58
                                      פנסיה                          33
פסגות קופות גמל ופנסיה בע"מ           גמל                            52

In [515]:
# report drill down
def report_sums_by_type(report_id):
    rep = all_holdings_final[all_holdings_final["report_id"] == report_id]
    print("total sum by holdings: {:,.2f}".format(rep["שווי"].sum()))
    return pd.DataFrame(rep.groupby("holding_type")["שווי"].agg('sum').map('{:,.2f}'.format))
    
report_sums_by_type('2106364')

total sum by holdings: 36,592,810.71


,שווי
holding_type,
"אג""ח קונצרני","2,376,256.90"
אופציות,"109,149.74"
הלוואות,"1,740,223.23"
השקעות אחרות,"-9,451.97"
זכויות מקרקעין,"236,814.27"
חוזים עתידיים,"22,285.89"
כתבי אופציה,"13,314.01"
"לא סחיר - אג""ח קונצרני","336,962.83"
לא סחיר - אופציות,"5,186.98"


### discrepancies

* report 2101262: empty sheet "זכויות מקרקעין" - explains all the diff
* report 2101276: all holding_types sums are ok, the sum @summary sheet is slightly different for some reason

In [123]:
print(funds_df[funds_df["ProductNum"] == 2207])
totals[totals["report_id"] == '2101262']

        SystemName                         ParentCorpName ParentCorpLegalId  \
2101262      פנסיה  מגדל מקפת קרנות פנסיה וקופות גמל בע"מ         512237744   

        ProductNum                                 Name  \
2101262       2207  מגדל מקפת אישית  למקבלי קצבה קיימים   

                             ShortName  StatusDate ReportPeriodDesc  
2101262  מקפת אישית למקבלי קצבה קיימים  29/08/2021     2021 רבעון 2  


,asset,sum,pct,report_id,pct_num,sum_num
4065,"סה""כ סכום נכסי המסלול או הקרן",3.58272e+06,1,2101262,1.0,3.582724e+06


# looking for lost insurance funds

### Migdal

In [ ]:
migdal_ins = all_holdings_final[(all_holdings_final["ParentCorpLegalId"] == 520004896)]
migdal_by_fund = pd.DataFrame(migdal_ins.groupby(["ShortName", "ProductNum"])["שווי"].agg('sum'))
migdal_by_fund.reset_index().sort_values("ProductNum")

https://www.migdal.co.il/about/investments/assets - there are 4 more funds not included in the reports site:
* מגדל משתתף ברווחים - קרן ח - b010_p221_520004896
* מגדל משתתף ברווחים - קרן ט - 520004896_b011_p221
* מגדל משתתף ברווחים - קרן י - b012_p221_520004896
* מגדל משתתף קרן י החדשה - b013_p221_520004896

Manually downloaded, added to the directory.

In [ ]:
pd.DataFrame(all_holdings_final[
    (all_holdings_final["ParentCorpLegalId"].isnull()) |
    (all_holdings_final["ParentCorpLegalId"] == 520004896)
].groupby("report_id")['שווי'].agg('sum').map('{:,.2f}'.format))

### Menora missing reports

In [130]:
pd.DataFrame(all_holdings_final[all_holdings_final["ParentCorpName"] == 'מנורה מבטחים ביטוח בע"מ'
].groupby("report_id")['שווי'].agg('sum').map('{:,.2f}'.format))

,שווי
report_id,
520042540_b101_p221,"279,185.82"
520042540_b13029_p221,"41,202.44"
520042540_b18011_p221,"518,784.98"
520042540_b18012_p221,"24,276,170.27"
520042540_b180_p221,"308,245.12"
520042540_b184_p221,"101,980.17"
520042540_b41_p221,"117,905.64"
520042540_b42_p221,"1,200,448.44"
520042540_b43_p221,"31,045.79"


In [ ]:
# adding sums of 2 migrated funds (data for these 2 is 1 quarter behind)
99576.34 + 531016.24 + all_holdings_final[all_holdings_final["ParentCorpName"] == 'מנורה מבטחים ביטוח בע"מ']["שווי"].sum()

Conclusion: there seems to be missing reports for Menora. Check again next quarter.

### הפניקס ביטוח - דוחות חסרים

In [ ]:
pd.DataFrame(
    all_holdings_final[all_holdings_final["ParentCorpLegalId"] == 520023185].groupby(["report_id", "ShortName", "ProductNum"])['שווי'].agg('sum').map('{:,.2f}'.format)
).sort_values("ProductNum", ascending=False)

added 2 missing reports: 7012, 7011, according to website: https://www.fnx.co.il/investors-relations-hebrew/investment-information-reports/reshimat-nichsey-hamevateah

### הראל ביטוח - דוחות חסרים

In [ ]:
pd.DataFrame(
    all_holdings_final[all_holdings_final["ParentCorpLegalId"] == 520004078].groupby(["report_id", "ShortName", "ProductNum"])['שווי'].agg('sum').map('{:,.2f}'.format)
).sort_values("ProductNum", ascending=False)

Manually added missing reports: 259010 259011 259012 259013
from website: https://www.harel-group.co.il/about/harel-group/harel/mesthtefet/Pages/list-of-assets.aspx

### כלל ביטוח - כל הדוחות חסרים

downloaded from their site: https://www.clalbit.co.il/aboutclalinsurance/generalreports/

## הלמן - פנסיה וגמל
?גמל - הבדל בסכום, האם כתוצאה מדוחות חסרים
<br>
₪16,136,063 בדוח מסכם
16,132,489.84 לפי סכום דוחות מסלול

הבדלים קטנים, לא מצאתי הסבר.
אולי הוציאו דוחות בשעות שונות של היום?

In [ ]:
16136063 - 16132489.84

In [ ]:
helman = all_holdings_final[(all_holdings_final["ParentCorpName"].str.startswith("הלמן")) &
                             (all_holdings_final["SystemName"] == "פנסיה")
                            ]
helman_by_report = pd.DataFrame(helman.groupby(["ProductNum", "ShortName"])["שווי"].agg('sum').map('{:,.2f}'.format)).reset_index()
helman_by_report

## מגדל פנסיה

In [129]:
migdal_p = all_holdings_final[(all_holdings_final["ParentCorpName"].str.startswith("מגדל")) &
                             (all_holdings_final["SystemName"] == "פנסיה")
                            ]
migdal_p_report = pd.DataFrame(migdal_p.groupby(["ProductNum", "ShortName"])["שווי"].agg('sum').map('{:,.2f}'.format)).reset_index()
migdal_p_report

,ProductNum,ShortName,שווי
0,2102,מקפת אישית כללי,"62,607,109.71"
1,2112,מגדל מקפת אישית הלכה,"1,132,152.25"
2,2142,מגדל מקפת אישית מניות,"2,067,624.37"
3,2143,מגדל מקפת אישית שקלי טווח קצר,"169,198.50"
4,2144,"מגדל מקפת אישית אג""ח","271,769.97"
5,2145,מקפת משלימה כללי,"838,344.64"
6,2146,מקפת משלימה מניות,"49,685.17"
7,2147,מקפת משלימה שקלי טווח קצר,"2,528.74"
8,2148,"מקפת משלימה אג""ח","5,041.36"
9,2149,מקפת משלימה הלכה,"5,580.40"
